In [ ]:
##########################################################################
# 인스타 그램의 이미지와 해쉬태그 수집하기
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import time
import sys
#import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야
import xlwt # excel 형태로 저장하기 위해 필요함. pip install xlwt 해야 사용가능함
import os
import random
import unicodedata 

s_time = time.time( )

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   Spiderware version 1.2.0 (2019-04-26) " )
print("   Copyright (C) 2019 Gachilabs for Data Scisence")
print("   Platform: x86_64-w64-mingw32/x64 (64-bit)")
print("=" *70)
print("   이 크롤러는 인스타그램의 사진과 해쉬태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 숙대맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 30)
f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
        f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
img_dir = f_dir+s+'-'+query_txt2+'-'+query_txt
os.makedirs(img_dir)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

In [5]:
# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(5)

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(3)

time.sleep(2)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]/div').click()

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)


i = 1
while (i <= real_cnt):
      scroll_down(driver) 
      i += 1

# Step 6. 전체 게시물의 원본 URL 추출하기
item=[]
count = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

all = soup.find('article','KC1QD').find_all('a')

for i in all:    
    url = i['href']
    item.append(url)  
    count += 1
    
    if count == cnt :
        break

# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 이미지 건수 세기
#hash_txt = []    # 해쉬 태그 저장 
file_no = 1      # 이미지 파일 번호 지정
img_src2=[]      # 이미지의 URL 저장
#img_path2 = []   # 이미지 파일의 저장 경로 저장

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    f = open(f_name, 'a',encoding='UTF-8')
        
    # 해당 페이지의 태그 수집
    tags = soup.find('div','EtaWk')
    
    try :
        tags_1 = tags.find_all('a')
    except :
        print('')
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)
            
            for i in tags_2 :
                if i[0:1]=='#' :
                    #hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    #print(hash_txt)
    
    # 해당 페이지의 이미지 수집
    imgs = soup.find('div','KL4Bh')
    try :
        img_src1=imgs.find('img')['src']
    except :
        continue
    else :
        img_name = img_dir +'\\'+ str(file_no)+'.jpg'
        urllib.request.urlretrieve(img_src1,img_name)       
        time.sleep(1)
        
        print("현재 %s 번째 게시물까지 저장완료했습니다=======" %file_no)
        print("\n")

        if file_no == cnt :
                break
        file_no += 1

        
e_time = time.time( )
t_time = e_time - s_time
total_count = file_no - 1

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %total_count)
print("파일 저장 경로: %s 입니다" %img_dir)
print("=" *70)

driver.close( )



요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^


#광고입니다
#강남맛집
#신사동맛집
#압구정맛집
#강남맛집추천
#레스토랑추천
#강남데이트
#스테이크맛집
#인생샷명소
#브런치맛집
#분위기깡패
현재 1 번째 게시물까지 저장완료했습니다=======


#닭갈비전
#강남역맛집
#강남맛집
#신논현맛집
#신논현역맛집
현재 2 번째 게시물까지 저장완료했습니다=======


#반룡산
#선릉역맛집
#대치동맛집
#가릿국밥
#회냉면
#이북음식
#퐈나짱_강남구
현재 3 번째 게시물까지 저장완료했습니다=======


#크래프트아일랜드강남역점
#강남역맛집
#강남맛집추천
#강남역술집추천
#강남역브런치
#강남술집
#강남맥주집
#잠실맛집추천
#방이동맛집추천
#송리단길맛집추천
#성남맛집
#문정동맛집
#위례맛집
현재 4 번째 게시물까지 저장완료했습니다=======


#강남역맛집
#강남맛집추천
#강남맛집베스트
#강남맛집투어
#일식맛집
#강남맛집탕방
#강남일식집
#강남역밥집
현재 5 번째 게시물까지 저장완료했습니다=======


#강남역맛집
#미도인
#강남맛집
#미도인강남
#강남데이트
#강남역스테이크
#강남맛집추천
#강남핫플레이스
#강남역데이트
현재 6 번째 게시물까지 저장완료했습니다=======


#doslocosrestaurant
#doslocos
#도스로코스
#강남맛집추천
#강남맛집
#신논현맛집
#타코
#케사디야
#멕시코음식
#멕시코맛집
#멕시코현지음식
#강남역먹자골목
#냠냠
#먹스타그램
#jmt
#hagamospaís
#mexico
#independenceday
현재 7 번째 게시물까지 저장완료했습니다=======


#강남역맛집
#강남맛집추천
#강남맛집베스트
#강남맛집투어
#일식맛집
#강남맛집탕방
#강남일식집
#강남역밥집
현재 8 번째 게시물까지 저장완료했습니다=======


#강남역맛집
#미도인강남
#미도인
#강남맛집
#미도인강남
#강남데이트
#강남역스테이크
#강남맛집추천
#강남핫플레이스
#강남역데이트
현재 9 번째 게시물까지 저장완료했습니다=======

In [4]:
print(img_name)

c:\temp\2020-09-20-22-14-51-인스타그램-강남맛집추천45.jpg
